# Train CIFAR with the `policy` module

Let's install the latest version of Poutyne (if it's not already) and import all the needed packages.

In [1]:
%pip install --upgrade poutyne

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import resnet18

from poutyne import Model, OptimizerPolicy, one_cycle_phases

## Training constant
But first, let's set the training constants, the CUDA device used for training if one is present, we set the batch size (i.e. the number of elements to see before updating the model) and the number of epochs (i.e. the number of times we see the full dataset).

In [2]:
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

batch_size = 1024
epochs = 5

# Load the data

In [3]:
_mean = [0.485, 0.456, 0.406]
_std = [0.229, 0.224, 0.225]


train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(.3, .3, .3),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])

In [4]:
root = "datasets"
train_ds = datasets.CIFAR10(root, train=True, transform=train_transform, download=True)
val_ds = datasets.CIFAR10(root, train=False, transform=val_transform, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_dl = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8
)
val_dl = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False,
    num_workers=8
)

# The model
We'll train a simple `resNet-18` network.
This takes a while without GPU but is pretty quick with GPU.

In [6]:
def get_module():
    model = resnet18(pretrained=False)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    model.fc = nn.Linear(512, 10)
    return model


# Training without the `policies` module

In [7]:
pytorch_network = get_module()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pytorch_network.parameters(), lr=0.01)

model = Model(
    pytorch_network,
    optimizer,
    criterion,
    batch_metrics=["acc"],
    device=device,
)

history = model.fit_generator(
    train_dl,
    val_dl,
    epochs=epochs,
)

Epoch: 1/5 Train steps: 49 Val steps: 10 8.36s loss: 2.117263 acc: 23.430000 val_loss: 1.857028 val_acc: 33.690000
Epoch: 2/5 Train steps: 49 Val steps: 10 8.35s loss: 1.797844 acc: 34.568000 val_loss: 1.663795 val_acc: 39.050000
Epoch: 3/5 Train steps: 49 Val steps: 10 8.42s loss: 1.643785 acc: 40.474000 val_loss: 1.546482 val_acc: 44.070000
Epoch: 4/5 Train steps: 49 Val steps: 10 8.38s loss: 1.550369 acc: 43.554000 val_loss: 1.469080 val_acc: 46.690000
Epoch: 5/5 Train steps: 49 Val steps: 10 8.39s loss: 1.481794 acc: 46.146000 val_loss: 1.425816 val_acc: 48.110000


# Training with the `policies` module

In [8]:
steps_per_epoch = len(train_dl)
steps_per_epoch

49

In [9]:
pytorch_network = get_module()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pytorch_network.parameters(), lr=0.01)

model = Model(
    pytorch_network,
    optimizer,
    criterion,
    batch_metrics=["acc"],
    device=device,
)

policy = OptimizerPolicy(
    one_cycle_phases(epochs * steps_per_epoch, lr=(0.01, 0.1, 0.008)),
)
history = model.fit_generator(
    train_dl,
    val_dl,
    epochs=epochs,
    callbacks=[policy],
)

Epoch: 1/5 Train steps: 49 Val steps: 10 8.49s loss: 1.864204 acc: 32.568000 val_loss: 1.752385 val_acc: 41.630000
Epoch: 2/5 Train steps: 49 Val steps: 10 8.48s loss: 1.403389 acc: 49.632000 val_loss: 1.403605 val_acc: 49.820000
Epoch: 3/5 Train steps: 49 Val steps: 10 8.64s loss: 1.172077 acc: 58.390000 val_loss: 1.093589 val_acc: 61.370000
Epoch: 4/5 Train steps: 49 Val steps: 10 8.55s loss: 0.983761 acc: 65.396000 val_loss: 0.971778 val_acc: 65.950000
Epoch: 5/5 Train steps: 49 Val steps: 10 8.53s loss: 0.826081 acc: 70.952000 val_loss: 0.898928 val_acc: 68.810000
